<a href="https://colab.research.google.com/github/leonejesus/colab_keras_face_spoof_train/blob/main/spoof_keras_train_tl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd drive/My\ Drive/Colab_Notebooks/

/content/drive/My Drive/Colab_Notebooks


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import Model, layers

model_name = "rufs"

batch_size=64

# create generator
datagen = ImageDataGenerator(horizontal_flip=True, preprocessing_function=preprocess_input)
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory('spoof_datasets/'+model_name+'_images/train', class_mode='categorical', batch_size=batch_size, target_size=(224,224))
val_it = datagen.flow_from_directory('spoof_datasets/'+model_name+'_images/val', class_mode='categorical', target_size=(224,224))

Found 696 images belonging to 2 classes.
Found 167 images belonging to 2 classes.


In [ ]:
conv_base = ResNet50(
    include_top=False,
    weights='imagenet',
    classes=2
    )

94773248/94765736 [==============================] - 0s 0us/step


In [ ]:
#for layer in conv_base.layers:
#    layer.trainable = False

In [ ]:
x = conv_base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='sigmoid', name = "extractor")(x) 
predictions = layers.Dense(2, activation='softmax')(x)
model = Model(conv_base.input, predictions)

In [ ]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc']
              )

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_filepath = "./"+model_name+"_results/models/res50_tl.{epoch:02d}-{val_acc:.2f}_2.h5"
#checkpoint_filepath = "./res50_tl.hdf5"
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
                                            save_weights_only=False,
                                            monitor='val_acc',
                                            mode='max',
                                            save_best_only=True
                                            )

In [ ]:
history = model.fit(train_it,
                    steps_per_epoch=len(train_it.filepaths) // batch_size,
                    epochs=30,
                    validation_data=val_it,
                    validation_steps=len(val_it.filepaths) // batch_size,
                    callbacks=[model_checkpoint_callback]
                    )

Epoch 1/30
10/10 [==============================] - 192s 18s/step - loss: 1.1663 - acc: 0.6072 - val_loss: 0.9284 - val_acc: 0.5312
Epoch 2/30
10/10 [==============================] - 17s 2s/step - loss: 0.4757 - acc: 0.7793 - val_loss: 0.6727 - val_acc: 0.5781
Epoch 3/30
10/10 [==============================] - 12s 1s/step - loss: 0.4048 - acc: 0.8125 - val_loss: 0.7897 - val_acc: 0.4219
Epoch 4/30
10/10 [==============================] - 10s 965ms/step - loss: 0.2391 - acc: 0.8954 - val_loss: 3.1467 - val_acc: 0.5000
Epoch 5/30
10/10 [==============================] - 8s 746ms/step - loss: 0.1956 - acc: 0.9209 - val_loss: 3.6240 - val_acc: 0.3750
Epoch 6/30
10/10 [==============================] - 7s 735ms/step - loss: 0.1678 - acc: 0.9394 - val_loss: 2.5697 - val_acc: 0.5312
Epoch 7/30
10/10 [==============================] - 7s 730ms/step - loss: 0.1190 - acc: 0.9621 - val_loss: 3.1536 - val_acc: 0.4062
Epoch 8/30
10/10 [==============================] - 7s 713ms/step - loss: 0.070

In [ ]:
# from keras.utils.vis_utils import plot_model
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)